In [2]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
print('Libraries Imported')

Libraries Imported


## 1. Preliminary exploratory data analysis

In [3]:
# OWID CO2 Dataset - Obtain from: https://ourworldindata.org/co2-and-other-greenhouse-gas-emissions
data = pd.read_csv('owid-co2-data.csv')
data.head(2)

,country,year,iso_code,population,gdp,cement_co2,cement_co2_per_capita,co2,co2_growth_abs,co2_growth_prct,...,share_global_cumulative_oil_co2,share_global_cumulative_other_co2,share_global_flaring_co2,share_global_gas_co2,share_global_oil_co2,share_global_other_co2,total_ghg,total_ghg_excluding_lucf,trade_co2,trade_co2_share
0,Afghanistan,1949,AFG,7624058.0,NaN,NaN,NaN,0.015,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,1950,AFG,7752117.0,9.421400e+09,NaN,NaN,0.084,0.07,475.0,...,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN


In [4]:
print(f'The dataset contains {data.shape[0]} rows and {data.shape[1]} columns.')

The dataset contains 26008 rows and 60 columns.


In [5]:
# Selecting the columns that we need for the analysis.
list(data.columns)

['country',
 'year',
 'iso_code',
 'population',
 'gdp',
 'cement_co2',
 'cement_co2_per_capita',
 'co2',
 'co2_growth_abs',
 'co2_growth_prct',
 'co2_per_capita',
 'co2_per_gdp',
 'co2_per_unit_energy',
 'coal_co2',
 'coal_co2_per_capita',
 'consumption_co2',
 'consumption_co2_per_capita',
 'consumption_co2_per_gdp',
 'cumulative_cement_co2',
 'cumulative_co2',
 'cumulative_coal_co2',
 'cumulative_flaring_co2',
 'cumulative_gas_co2',
 'cumulative_oil_co2',
 'cumulative_other_co2',
 'energy_per_capita',
 'energy_per_gdp',
 'flaring_co2',
 'flaring_co2_per_capita',
 'gas_co2',
 'gas_co2_per_capita',
 'ghg_excluding_lucf_per_capita',
 'ghg_per_capita',
 'methane',
 'methane_per_capita',
 'nitrous_oxide',
 'nitrous_oxide_per_capita',
 'oil_co2',
 'oil_co2_per_capita',
 'other_co2_per_capita',
 'other_industry_co2',
 'primary_energy_consumption',
 'share_global_cement_co2',
 'share_global_co2',
 'share_global_coal_co2',
 'share_global_cumulative_cement_co2',
 'share_global_cumulative_co2',

In [6]:
# Selecting the columns that we need for the analysis.
co2_df = data[['country', 'year', 'iso_code', 'co2']]

In [7]:
# Checking dtypes
display(co2_df.dtypes)

country      object
year          int64
iso_code     object
co2         float64
dtype: object

In [8]:
# Checking for null data
co2_df.isnull().sum()

country        0
year           0
iso_code    4095
co2         1338
dtype: int64

In [9]:
# Checking duplicated rows
data.duplicated().sum()

0

In [10]:
# How many countries are in the dataset?
co2_df.groupby('country')['co2'].sum()

country
Afghanistan              192.851
Africa                 47610.750
Albania                  285.504
Algeria                 4587.132
Andorra                   15.072
                        ...     
Wallis and Futuna          0.508
World                1696524.172
Yemen                    624.651
Zambia                   247.985
Zimbabwe                 779.382
Name: co2, Length: 247, dtype: float64

### Seems like there are more countries/regions than expected. Let's filter these countries with another dataset

## 2. Small Cleansing

In [11]:
# Countries dataset: https://www.kaggle.com/datasets/fernandol/countries-of-the-world?resource=download
countries_df = pd.read_csv('countries of the world.csv')
countries_df.head(2)

,Country,Region,Population,Area (sq. mi.),Pop. Density (per sq. mi.),Coastline (coast/area ratio),Net migration,Infant mortality (per 1000 births),GDP ($ per capita),Literacy (%),Phones (per 1000),Arable (%),Crops (%),Other (%),Climate,Birthrate,Deathrate,Agriculture,Industry,Service
0,Afghanistan,ASIA (EX. NEAR EAST),31056997,647500,"48,0","0,00","23,06","163,07",700.0,"36,0","3,2","12,13","0,22","87,65",1,"46,6","20,34","0,38","0,24","0,38"
1,Albania,EASTERN EUROPE,3581655,28748,"124,6","1,26","-4,93","21,52",4500.0,"86,5","71,2","21,09","4,42","74,49",3,"15,11","5,22","0,232","0,188","0,579"


In [12]:
countries_df = countries_df.rename(str.lower, axis = 'columns')

In [13]:
# Small cleaning
countries_list = [i.strip(' ') for i in countries_df['country']]

In [67]:
# Masking and creating a clean dataframe with only countries | not regions
mask = (co2_df['country'].isin(countries_list)) &  (co2_df['co2'] > 0)
clean_df = co2_df[mask]

In [100]:
# Top 10 countries all history that emit the most CO2
top = list(clean_df.groupby('country')['co2'].sum().reset_index().sort_values(by='co2', ascending = False)['country'][:10])

## 3. Visualization

In [101]:
# Line chart. Top 10 countries that emit the most CO2 in history
mask = clean_df['country'].isin(top)
px.line(clean_df[mask], x = 'year', y = 'co2', color = 'country', title = 'Top 5 countries that emit the most CO2: History')

After the industrial revolution, there was a fairly linear growth in CO2 emissions by the United States until 2005, after which there was a decrease until the current year. China increases its emissions around 1961, with an exponential growth curve until the current year, it does not seem to decrease. After the Second World War, Russia shows a growth in its emissions until 1991, after this year it shows a constant behavior of emissions. After 1985 India shows a growth in its emissions that decreases for the period 2019-2020.

In [16]:
# Masking for year 2020
mask_2020 = clean_df['year'] == 2020
clean_df_2020  = clean_df[mask_2020]
clean_df_2020 = pd.DataFrame(clean_df_2020.groupby('country')['co2'].sum()).reset_index().sort_values(by='co2', ascending = False)

In [17]:
# Masking for year 2000
mask_2000 = clean_df['year'] == 2000
clean_df_2000 = clean_df[mask_2000]
clean_df_2000 = pd.DataFrame(clean_df_2000.groupby('country')['co2'].sum()).reset_index().sort_values(by='co2', ascending = False)

In [41]:
# Masking fo year 2000 and 2020
mask_years = (clean_df['year'] == 2000) | (clean_df['year'] == 2020)
clean_df_years = clean_df[mask_years].sort_values(by = 'co2', ascending = False)

In [143]:
# Barchart
fig = go.Figure(data = [
    go.Bar(
        name = '2020',
        x = clean_df_2020['country'][:5], 
        y = clean_df_2020['co2'][:5],
        marker = dict(color = '#f5c064'),
        text = clean_df_2020['co2'][:5],
        textposition = "outside"
        ),

    go.Bar(
    name = '2000',
    x = clean_df_2000['country'][:5], 
    y = clean_df_2000['co2'][:5],
    marker = dict(color = '#6e0000'),
    text = clean_df_2000['co2'][:5],
    textposition = "outside"    
    )
]
)
fig.update_layout(barmode = 'group', title = 'CO2 Gas Emissions Ranked 5 Countries: 2000 vs 2020',
                  width = 900, height = 500,
                  paper_bgcolor = '#F2F3FD')
fig.show()

It can be apreciated that the top 5 countries that emitted the most CO2 in the year 2000 are the same countries that emit CO2 in the year 2020. This goes hand in hand with the exponential growth in industrial sectors, fossil waste management, overpopulation, natural resources, livestock and other factors.

In [112]:
# Exporting new data for power BI visualization
clean_df.to_csv('countries_co2_history.csv', index = False)